On va considérer un graphe ponderé donné par des listes d'adjacence sous la forme `(voisin, poids)`.

In [6]:
type pond_adj = (int * int) list array

type pond_adj = (int * int) list array

Le graphe d'exemple vu en classe est le suivant :

In [1]:
let gtest = [|
    [ (1, 8); (2,5)];
    [ (0, 8); (4, 18); (3, 2); (1, 10) ];
    [ (0, 5); (1, 10); (3, 3); (5, 16) ];
    [ (1, 2); (2, 3); (4, 12); (5,30); (6, 14) ];
    [ (1, 18); (3, 12); (6, 4) ];
    [ (3, 30); (2, 16); (6, 26) ];
    [ (4, 4); (3, 14); (5, 26) ]
    |]

val gtest : (int * int) list array =
  [|[(1, 8); (2, 5)]; [(0, 8); (4, 18); (3, 2); (1, 10)];
    [(0, 5); (1, 10); (3, 3); (5, 16)];
    [(1, 2); (2, 3); (4, 12); (5, 30); (6, 14)]; [(1, 18); (3, 12); (6, 4)];
    [(3, 30); (2, 16); (6, 26)]; [(4, 4); (3, 14); (5, 26)]|]

Pour utiliser des estimations de distance, on va considérer un tableau d'entiers. Afin de représenter l'infini on utilise juste `max_int` le plus grand entier représentable.

In [51]:
let inf = max_int (* on utilise max_int comme valeur infinie *)

val inf : int = 2147483647

Écrire une fonction qui prend en entrée un graphe et un sommet et intialise l'estimation de distance.

In [52]:
let initialise_estimation g s =
    let a = Array.make (Array.length g) inf
    in a.(s) <- 0;
    a

val initialise_estimation : 'a array -> int -> int array = <fun>

Écrire une fonction qui détermine si une arête `x -> y` de poids `w` est tendue pour l'estimation `d` :

In [53]:
let est_tendue d x y w =
    d.(y) > d.(x) + w

val est_tendue : int array -> int -> int -> int -> bool = <fun>

Écrire une fonction qui relache l'arête.

In [54]:
let relache d x y w =
    d.(y) <- d.(x) + w 

val relache : int array -> int -> int -> int -> unit = <fun>

On va réaliser l'algorithme de Bellman-Ford qui relache toutes les arêtes tendues en effectuant $|S|$ de fois l'opération consistant à parcourir chaque arête et à la relacher si elle est tendue. Votre fonction devra renvoyer l'estimation de distance finale qui contiendra la plus petite distance depuis `s`.

In [55]:
let bellman_ford g s =
    let d =  initialise_estimation g s in
    let l = Array.length g in
   for _ = 1 to l do
        for x = 0 to l-1 do
            List.iter (fun (y,w)-> if est_tendue d x y w then relache d x y w) g.(x)
        done
    done;
    d

val bellman_ford : (int * int) list array -> int -> int array = <fun>

In [56]:
bellman_ford gtest 0

- : int array = [|0; 8; 5; 8; 20; 21; 22|]

In [57]:
assert (bellman_ford gtest 0 = [|0; 8; 5; 8; 20; 21; 22|]);
"OK"

- : string = "OK"

On veut mainteant que votre fonction renvoie un couple `(d, pred)` où `d` est toujours la distance mais `pred : int option array` est un tableau indiquant le prédecesseur dans un plus court chemin.

**Note** réflechissez sur un papier au besoin.

In [69]:
let bellman_ford_pred g s =
    let d =  initialise_estimation g s in
    let pred = Array.make (Array.length g) None in
    let l = Array.length g in
   for _ = 1 to l do
        for x = 0 to l-1 do
            List.iter (fun (y,w)-> if est_tendue d x y w then relache d x y w) g.(x);
            pred.(x) <- Some (snd (List.fold_left min (inf,69) 
                (List.map (fun (y,w) -> d.(y) + w ,y) g.(x))))
        done
    done;
    pred.(s) <- None;
    d,pred

val bellman_ford_pred :
  (int * int) list array -> int -> int array * int option array = <fun>

In [68]:
bellman_ford_pred gtest 0

- : int array * int option array =
([|0; 8; 5; 8; 20; 21; 22|],
 [|Some 2; Some 0; Some 0; Some 2; Some 3; Some 2; Some 3|])

In [70]:
assert (bellman_ford_pred gtest 0 = ([|0; 8; 5; 8; 20; 21; 22|],
 [|None; Some 0; Some 0; Some 2; Some 3; Some 2; Some 3|]));
"OK"

- : string = "OK"

Écrire une fonction qui prend une liste de sommets `l` et une estimation `d` et renvoie un couple `(i, l')` 
où `i` est l'indice du minimum de `d` dans `l` et `l'` la liste déduite de `l` en supprimant `i` (et en préservant l'ordre).

In [75]:
let extrait_min l d =
    let _,ind = List.fold_left min  (inf,69) (List.map (fun x -> d.(x),x) l)
    in ind,List.filter ((<>) ind) l

val extrait_min : int list -> int array -> int * int list = <fun>

In [72]:
extrait_min [1;3;4] [|0;4;2;3;5|]

- : int * int = (3, 3)

In [76]:
assert (extrait_min [1;3;4] [|0;4;2;3;5|] = (3, [1; 4]));
"OK"

- : string = "OK"

En déduire une implémentation naïve de Dijkstra où les sommets à visiter sont maintenus dans une `int list ref`.
Cette fonction aura la même interface que `bellman_ford_pred`.

In [84]:
let dijkstra_naif g s =
    let d = initialise_estimation g s in
    let a_traiter = ref (List.init (Array.length g) Fun.id) in
    let pred = Array.make (Array.length g) None in
    while !a_traiter <> [] do
        let x, to_treat = extrait_min !a_traiter d in
        List.iter (fun (y,w) -> 
            if est_tendue d x y w then 
            (relache d x y w; pred.(y) <- Some x)) g.(x);
        a_traiter := to_treat
    done;
    d,pred

val dijkstra_naif :
  (int * int) list array -> int -> int array * int option array = <fun>

In [85]:
dijkstra_naif gtest 0

- : int array * int option array =
([|0; 8; 5; 8; 20; 21; 22|],
 [|None; Some 0; Some 0; Some 2; Some 3; Some 2; Some 3|])

In [86]:
assert (dijkstra_naif gtest 0 = ([|0; 8; 5; 8; 20; 21; 22|],
 [|None; Some 0; Some 0; Some 2; Some 3; Some 2; Some 3|]));
"OK"

- : string = "OK"

On va réutiliser les modules `tas.ml` et `filePriorite.ml` vus dans un TP précédent. Pour pouvoir les utiliser dans Capytale, on les a explicitement mis sous la forme d'un module.

Dans cette question, on vous demande juste d'écrire le code de la fonction `remplace` qui va remplacer une valeur par une autre dans le tas à un indice donné et **réparer le tas ensuite**.

In [90]:
module Tas = struct 

    type 'a t = {
    elements : 'a array;
    mutable taille : int
    }

    let cree taille_max defaut =
        {
            elements = Array.make taille_max defaut;
            taille = 0
        }

    let est_vide tas = tas.taille = 0

    let taille tas = tas.taille

    let pere a =
        if a = 0
        then failwith "racine"
        else (a+1)/2 - 1

    let gauche a = 2*(a+1) - 1
    let droite a = 2*(a+1)

    let minimum tas = tas.elements.(0)

    let nth tas i = tas.elements.(i)

    let swap v x y =
        let t = v.(x) in
        v.(x) <- v.(y);
        v.(y) <- t

    let ajoute x tas =
        let pos = ref tas.taille in
        tas.elements.(tas.taille) <- x;
        tas.taille <- tas.taille + 1;
        while !pos > 0 && nth tas (pere !pos) > nth tas !pos do
            let p = pere !pos in
            swap tas.elements p !pos;
            pos := p
        done

    let min_pere_fils tas a =
        let min_courant = ref a in

        if tas.taille > gauche a
            && nth tas (gauche a) < nth tas !min_courant
        then min_courant := gauche a;
        if tas.taille > droite a
            && nth tas (droite a) < nth tas !min_courant
        then min_courant := droite a;
        !min_courant

    let extrait_minimum tas =
        assert (tas.taille > 0);
        tas.taille <- tas.taille - 1;
        let x = tas.elements.(0) in
        if tas.taille > 0
        then begin
            let pos = ref 0 in
            tas.elements.(0) <- tas.elements.(tas.taille);
            while min_pere_fils tas !pos <> !pos do
                let min_fils = min_pere_fils tas !pos in
                swap tas.elements !pos min_fils;
                pos := min_fils
            done
        end;
        x
        
    let remplace tas i p =
        tas.elements.(i) <- p;
        ajoute (extrait_minimum tas) tas;

    exception Trouve of int
    let recherche tas f =
        try
            for i = 0 to tas.taille - 1 do
                if f (nth tas i)
                then raise (Trouve i)
            done;
            raise Not_found
        with Trouve i -> i
end

module Tas :
  sig
    type 'a t = { elements : 'a array; mutable taille : int; }
    val cree : int -> 'a -> 'a t
    val est_vide : 'a t -> bool
    val taille : 'a t -> int
    val pere : int -> int
    val gauche : int -> int
    val droite : int -> int
    val minimum : 'a t -> 'a
    val nth : 'a t -> int -> 'a
    val swap : 'a array -> int -> int -> unit
    val ajoute : 'a -> 'a t -> unit
    val min_pere_fils : 'a t -> int -> int
    val extrait_minimum : 'a t -> 'a
    val remplace : 'a t -> int -> 'a -> unit
    exception Trouve of int
    val recherche : 'a t -> ('a -> bool) -> int
  end

In [91]:
module FileMinPriorite = struct 
    type ('a, 'b) t = ('b * 'a) Tas.t

    let cree taille_max defval defprio =
        Tas.cree taille_max (defprio, defval)
        
    let taille file = file.Tas.taille

    let ajoute file x p =
        Tas.ajoute (p, x) file

    let retire file =
        let _, x = Tas.extrait_minimum file in
        x
        
    let diminue_priorite file x p =
        let i = Tas.recherche file (fun (_, y) -> x = y) in
        Tas.remplace file i (p, x)
end

module FileMinPriorite :
  sig
    type ('a, 'b) t = ('b * 'a) Tas.t
    val cree : int -> 'a -> 'b -> ('b * 'a) Tas.t
    val taille : 'a Tas.t -> int
    val ajoute : ('a * 'b) Tas.t -> 'b -> 'a -> unit
    val retire : ('a * 'b) Tas.t -> 'b
    val diminue_priorite : ('a * 'b) Tas.t -> 'b -> 'a -> unit
  end

In [92]:
let t = FileMinPriorite.cree 10 0 0 in
FileMinPriorite.ajoute t 1 50;
FileMinPriorite.ajoute t 2 10;
FileMinPriorite.ajoute t 3 20;
FileMinPriorite.ajoute t 4 30;
FileMinPriorite.ajoute t 5 20;
FileMinPriorite.diminue_priorite t 5 5;
assert(FileMinPriorite.retire t = 5);
"OK"

- : string = "OK"

Reprendre alors l'implémentation de Dijkstra avec cette file de priorité.

In [100]:
let dijkstra g s =
    let d = initialise_estimation g s in
    let a_traiter = FileMinPriorite.cree (Array.length g) 0 0 in
    for i=Array.length g - 1  downto 0 do FileMinPriorite.ajoute a_traiter i (if i=s then 0 else inf)  done; 
    let pred = Array.make (Array.length g) None in
    while FileMinPriorite.taille a_traiter > 0 do
        let x = FileMinPriorite.retire a_traiter in
        List.iter (fun (y,w) -> 
            if est_tendue d x y w then 
            (relache d x y w; pred.(y) <- Some x; FileMinPriorite.diminue_priorite a_traiter y d.(y))) g.(x)
    done;
    d,pred

val dijkstra : (int * int) list array -> int -> int array * int option array =
  <fun>

In [101]:
dijkstra gtest 0

- : int array * int option array =
([|0; 8; 5; 8; 20; 21; 22|],
 [|None; Some 0; Some 0; Some 2; Some 3; Some 2; Some 3|])